<a href="https://colab.research.google.com/github/fabriciosantana/mcdia/blob/main/01-icd/assignments/02_analise_quantitativa_discursos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Análise Quantitativa dos Discursos do Senado (56ª Legislatura)

Este notebook examina o conjunto de discursos proferidos no Senado Federal durante a 56ª Legislatura (01/02/2019 a 31/01/2023).

Principais objetivos:
- compreender a composição do acervo (autores, partidos, datas e metadados)
- identificar padrões temporais de produção de discursos
- analisar o tamanho dos textos e a distribuição de temas (tipos de uso da palavra)
- gerar sumários numéricos e visuais que sirvam de base para estudos posteriores

## Inicializar e carregar dados

### Importar bibliotecas

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from datasets import load_dataset
from datasets import load_dataset_builder

pd.set_option('display.max_columns', None)
sns.set_theme(style='whitegrid', palette='tab10')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

### Carregar dataset

In [ ]:
#local do arquivo de dados no huggingface
DATASET_HF_REPO = "fabriciosantana/discursos-senado-legislatura-56"
DATA_FILE_HF = {"train": "data/full/discursos_2019-02-01_2023-01-31.parquet"}

ds_builder = load_dataset_builder(DATASET_HF_REPO)

ds_builder.info

### Converter dataset para dataframe

In [ ]:
dataset = load_dataset(DATASET_HF_REPO, data_files=DATA_FILE_HF)
df = dataset["train"].to_pandas()

df

### Analisar estrutura do dataframe

In [ ]:
print(f'Linhas: {df.shape[0]} | Colunas: {df.shape[1]}')
df.info()

## Analisar qualidade dos dados

### Visão geral dos dados

In [ ]:
overview = pd.DataFrame({
    'dtype': df.dtypes.astype(str),
    'missing': df.isna().sum()
}).sort_values('missing', ascending=False)
overview

### Métricas dos dados

In [ ]:
metrics = pd.Series({
    'Discursos': len(df),
    'Autores únicos': df['NomeAutor'].nunique(),
    'Partidos únicos': df['Partido'].nunique(),
    'UFs representadas': df['UF'].replace('', np.nan).nunique(dropna=True),
    'Funções de autor únicas': df['FuncaoAutor'].replace('', np.nan).nunique(dropna=True),
    'Discursos com resumo': int((df['Resumo'].str.len() > 0).sum()),
    'Discursos com indexação': int((df['Indexacao'].str.len() > 0).sum()),
    'Discursos com texto integral': int((df['TextoDiscursoIntegral'].str.len() > 0).sum())
}).to_frame('quantidade')
metrics

### Separar dados faltantes

In [ ]:
missing = (
    df.isna()
      .sum()
      .to_frame('faltantes')
      .assign(percentual=lambda s: (s['faltantes'] / len(df) * 100).round(2))
      .sort_values('faltantes', ascending=False)
)
missing.head(10)

## Analisar discursos por data
Criar campos para ano, mês e dia da semana

In [ ]:
#converte o tipo para datetime
df['Data'] = pd.to_datetime(df['Data'], errors='coerce')

#cria coluna para o ano
df['ano'] = df['Data'].dt.year

#cria coluna para o mês
df['mes'] = df['Data'].dt.to_period('M')

dias_map = {
    'Monday': 'Segunda',
    'Tuesday': 'Terça',
    'Wednesday': 'Quarta',
    'Thursday': 'Quinta',
    'Friday': 'Sexta',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}

#cria coluna para a semana
df['dia_semana'] = df['Data'].dt.day_name().map(dias_map)

df[["Data", "ano", "mes", "dia_semana"]].head()

### Discursos por ano

In [ ]:
discursos_por_ano = (df.groupby(["ano"])
                     .size()
                     .reset_index(name="qtd_discursos")
                    )

ax = discursos_por_ano.plot(x="ano", y="qtd_discursos", kind="bar", legend=False)
ax.set_title("Discursos por ano")
ax.set_xlabel("Ano")
ax.set_ylabel("Qtd. de discursos")

discursos_por_ano

### Discursos por mês

In [ ]:
discursos_por_mes = (
    df.dropna(subset=['mes'])
      .groupby('mes')
      .size()
      .rename('discursos')
      .sort_index()
)

fig, ax = plt.subplots()
sns.lineplot(x=discursos_por_mes.index.to_timestamp(), y=discursos_por_mes.values, marker='o', ax=ax)
ax.set(title='Volume mensal de discursos', xlabel='Mês', ylabel='Quantidade')
plt.xticks(rotation=45)
plt.tight_layout()
discursos_por_mes.tail()

### Discursos por dia da semana

In [ ]:
dias_semana = ['Segunda', 'Terça', 'Quarta', 'Quinta', 'Sexta']
discurso_por_dia_semana = (
    df.dropna(subset=['dia_semana'])
      .groupby('dia_semana')
      .size()
      .reindex(dias_semana, fill_value=0)
      .rename('discursos')
      .reset_index()
)
fig, ax = plt.subplots(figsize=(8, 5))
sns.barplot(
    data=discurso_por_dia_semana,
    x="dia_semana", y="discursos",
    order=dias_semana,
    color="C0"            # usa uma cor única (evita warning do seaborn)
)
ax.set(title='Distribuição de discursos por dia da semana', 
       xlabel='Dia da semana', 
       ylabel='Quantidade de discursos'
      )

plt.tight_layout()

discurso_por_dia_semana

### Discursos anuais por partido

In [ ]:
top_partidos_lista = (
    df['Partido']
    .dropna()
    .value_counts()
    .head(10)
    .index
    .tolist()
)
heatmap_data = (
    df[df['Partido'].isin(top_partidos_lista)]
      .dropna(subset=['ano'])
      .groupby(['ano', 'Partido'])
      .size()
      .unstack(fill_value=0)
      .reindex(columns=top_partidos_lista)
      .sort_index()
)
fig, ax = plt.subplots(figsize=(12, 6))
sns.heatmap(heatmap_data, cmap='Blues', linewidths=0.5, ax=ax)
ax.set(title='Participação anual por partido (top 10)', xlabel='Partido', ylabel='Ano')
plt.tight_layout()
heatmap_data


## Analisar discursos por autor, partido e UF

### Discursos por autor

In [ ]:
discursos_por_autor = (
    df.groupby('NomeAutor')
      .size()
      .sort_values(ascending=False)
      .head(10)
      .rename('discursos')
      .reset_index()
)
fig, ax = plt.subplots(figsize=(12, 6))
sns.barplot(data=discursos_por_autor, y='NomeAutor', x='discursos', hue='NomeAutor', ax=ax, palette='crest')
ax.set(title='Autores com mais discursos', xlabel='Quantidade de discursos', ylabel='')
plt.tight_layout()
discursos_por_autor

### Discursos por partido

In [ ]:
discursos_por_partido = (
    df.groupby('Partido')
      .size()
      .sort_values(ascending=False)
      .head(10)
      .rename('discursos')
      .reset_index()
)
fig, ax = plt.subplots(figsize=(12, 6))
sns.barplot(data=discursos_por_partido, x='Partido', y='discursos', ax=ax)
ax.set(title='Partidos com mais discursos', xlabel='Partido', ylabel='Quantidade de discursos')
plt.xticks(rotation=45)
plt.tight_layout()
discursos_por_partido

### Discursos por UF

In [ ]:
discursos_por_partido = (
    df['UF']
      .replace('', 'Não informado')
      .value_counts()
      .head(10)
      .rename('discursos')
      .reset_index()
      .rename(columns={'index': 'UF'})
)
fig, ax = plt.subplots(figsize=(10, 6))
sns.barplot(data=discursos_por_partido, x='discursos', y='UF', ax=ax)
ax.set(title='Estados com mais discursos registrados', xlabel='Quantidade de discursos', ylabel='UF')
plt.tight_layout()
discursos_por_partido

### Discursos por tipo de uso da palavra

In [ ]:
tipo_uso_palavra = (
    df['TipoUsoPalavra.Descricao']
      .replace('', 'Não informado')
      .value_counts()
      .head(10)
      .rename('discursos')
      .reset_index()
      .rename(columns={'index': 'Tipo de uso'})
)
fig, ax = plt.subplots(figsize=(12, 6))
sns.barplot(data=tipo_uso_palavra, x='discursos', y='TipoUsoPalavra.Descricao', ax=ax)
ax.set(title='Principais tipos de uso da palavra', xlabel='Quantidade de discursos', ylabel='')
plt.tight_layout()
tipo_uso_palavra

## Analisar texto dos discursos

Todos as colunas vazias devem conter uma string vazia

In [ ]:
text_columns = [
    'Resumo', 'Indexacao', 'TextoIntegral', 'TextoIntegralTxt',
    'TextoDiscursoIntegral', 'TipoUsoPalavra.Descricao', 'TipoUsoPalavra.Sigla',
    'TipoUsoPalavra.Codigo', 'TipoUsoPalavra.IndicadorAtivo',
    'Publicacoes.Publicacao', 'Apartes.Aparteante', 'CargoAutor',
    'OrgaoAutor', 'PaisAutor'
]
for col in text_columns:
    if col in df.columns:
        df[col] = df[col].fillna('').astype(str)

df[df['TextoDiscursoIntegral'] == '']

### Palavras e caracteres por discurso

In [ ]:

df['texto_len_palavras'] = df['TextoDiscursoIntegral'].str.split().str.len()
df['texto_len_caracteres'] = df['TextoDiscursoIntegral'].str.len()
df['resumo_len_palavras'] = df['Resumo'].str.split().str.len()

df[['id', 'texto_len_palavras', 'texto_len_caracteres', 'resumo_len_palavras']].head()

### Distribuição do tamanho dos discursos

In [ ]:
fig, ax = plt.subplots()
sns.histplot(df['texto_len_palavras'], bins=50, ax=ax)
ax.set(title='Distribuição do tamanho dos discursos (palavras)', xlabel='Número de palavras', ylabel='Frequência')
plt.tight_layout()

### Tamanho do discurso (palavras e caracteres)

In [ ]:
tamanho_discursos = pd.DataFrame({
    'palavras': df['texto_len_palavras'].describe().round(2),
    'caracteres': df['texto_len_caracteres'].describe().round(2)
})
tamanho_discursos

## Principais achados

- Os discursos se concentram entre 2019 e 2022, com picos mensais próximos às sessões de maior atividade legislativa.
- Autores como os líderes partidários respondem pelo maior volume de pronunciamentos, e PT, Podemos e MDB aparecem como partidos mais prolíficos.
- A maioria dos discursos possui texto integral disponível, com mediana de aproximadamente 464 palavras (≈2,8 mil caracteres) e cauda longa que indica pronunciamentos extensos.
- Tipos de uso da palavra relacionados a relatorias e lideranças dominam o plenário, reforçando o papel institucional desses cargos.
- A distribuição semanal mostra concentração entre terça e quinta-feira, dias tradicionalmente destinados às deliberações no Senado.